In [4]:
import starepandas
import geopandas
import sqlalchemy

# Data Prep

In [5]:
cities = ['Buenos Aires', 'Brasilia', 'Santiago', 
          'Bogota', 'Caracas', 'Sao Paulo', 'Bridgetown']

latitudes = [-34.58, -15.78, -33.45, 4.60, 10.48, -23.55, 13.1]
longitudes = [-58.66, -47.91, -70.66, -74.08, -66.86, -46.63, -59.62]
data =  {'City': cities, 
         'Latitude': latitudes, 'Longitude': longitudes}

cities = starepandas.STAREDataFrame(data)
stare = starepandas.stare_from_xy(cities.Longitude, cities.Latitude, level=27)
cities.set_stare(stare, inplace=True)
geom = geopandas.points_from_xy(cities.Longitude, cities.Latitude, crs='EPSG:4326')
cities.set_geometry(geom, inplace=True)

# To SQLite

In [7]:
db_path = 'data/test.sqlite' # Empty string for inmemory
uri = 'sqlite:///{db_path}'.format(db_path=db_path)
engine = sqlalchemy.create_engine(uri)
cities = geopandas.io.sql._convert_to_ewkb(cities, 'geometry', 4326)
cities.to_sql(name='cities', con=engine, if_exists='replace')

# To ScidDB

In [8]:
import scidbpy
db = scidbpy.db.DB(scidb_url='http://schiss.eri.ucsb.edu:8080/')

HTTPError: 502 Server Error: SciDB connection failed for url: http://schiss.eri.ucsb.edu:8080/new_session?admin=0

In [ ]:
schema = '<Latitude: float, stare: int64>[idx]'
records = cities[['Latitude', 'stare']].to_records(index=False)
a = db.input(schema, upload_data=records).store('cities')

In [ ]:
cities.dtypes

In [ ]:
query = 'scan({array})'.format(array='test')
db.iquery(query=query, fetch=True)

In [ ]:
query = "remove({array})".format(array=name)
db.iquery(query=query, fetch=False)